### First we will import all our data sctructure and visualization tools

In [1]:
import sys; sys.path.insert(0, "..") 
import os
import random
import src.visualization.utils as vis
from src.dataset.dataset import CustomDataset 
from src.dataset.utils import get_active_learning_datasets

### Lets load our data

In [2]:
dataset = CustomDataset(data_path="../data/NPY/volumes/",target_path="../data/NPY/labels/")
print("dataset length: {}".format(len(dataset)))
_ = get_active_learning_datasets(dataset)

dataset length: 3186
CP sample len 1496
NCP sample len 1218
Normal sample len 472
size train files: 990
size test files: 282
size val files: 141
size initial pool files: 3


### Lets plot some slices from a random volume

In [3]:
random_idx = random.sample(range(len(dataset)), 2)
n_slices = 10

for idx in random_idx:
    volume, label = dataset[idx]
    vis.plot_from_volume_tensor(volume,
                                label,
                                n_samples=n_slices,
                                n_cols=3, 
                                title=dataset.data_paths[idx])

ValueError: Sample larger than population or is negative

<Figure size 720x720 with 0 Axes>

### Now lets visualize a whole volume 

In [10]:
random_idx = random.randint(0, len(dataset))
print(dataset.data_paths[random_idx])
volume, label = dataset[random_idx]
vis.plot_volume(volume)

../data/NPY/volumes/vol_CP_3748_5692.npy


Output()